<a href="https://colab.research.google.com/github/LIWEIZHEN-111/ch2/blob/main/3_making_dataframes_from_api_requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/earthquakes.csv

!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/example_data.csv

!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/parsed.csv

!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/quakes.db

!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/tsunamis.csv

!mkdir data

!mv *.csv data

!mv *.db data

--2024-09-19 10:10:57--  https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/earthquakes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3515656 (3.4M) [text/plain]
Saving to: ‘earthquakes.csv’

earthquakes.csv     100%[===================>]   3.35M  --.-KB/s    in 0.08s   

2024-09-19 10:10:57 (41.0 MB/s) - ‘earthquakes.csv’ saved [3515656/3515656]

--2024-09-19 10:10:57--  https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/example_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.1

In [2]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [3]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [4]:
earthquake_json['metadata']

{'generated': 1726740661000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-08-19&endtime=2024-09-18',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.1',
 'count': 10064}

Each element in the JSON array `features` is a row of data for our dataframe.

In [5]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [6]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.89,
  'place': '18 km ENE of Honaunau-Napoopoo, Hawaii',
  'time': 1726616937740,
  'updated': 1726618967710,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/hv74468222',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=hv74468222&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 55,
  'net': 'hv',
  'code': '74468222',
  'ids': ',hv74468222,',
  'sources': ',hv,',
  'types': ',origin,phase-data,',
  'nst': 28,
  'dmin': 0.01065,
  'rms': 0.16,
  'gap': 92,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.9 - 18 km ENE of Honaunau-Napoopoo, Hawaii'},
 'geometry': {'type': 'Point',
  'coordinates': [-155.692666666667, 19.5091666666667, 7.21]},
 'id': 'hv74468222'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [7]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.89,"18 km ENE of Honaunau-Napoopoo, Hawaii",1726616937740,1726618967710,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv74468222,",",hv,",",origin,phase-data,",28.0,0.01065,0.16,92.0,ml,earthquake,"M 1.9 - 18 km ENE of Honaunau-Napoopoo, Hawaii"
1,1.44,"41 km SSE of Mammoth, Wyoming",1726616394600,1726666853220,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",uu80082806,",",uu,",",origin,phase-data,",10.0,0.08200,0.11,109.0,ml,earthquake,"M 1.4 - 41 km SSE of Mammoth, Wyoming"
2,0.71,"4 km SSE of The Geysers, CA",1726615773040,1726621766362,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc75063251,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",11.0,0.01508,0.14,132.0,md,earthquake,"M 0.7 - 4 km SSE of The Geysers, CA"
3,1.82,"4 km NNW of Boron, CA",1726615127700,1726679584350,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci40736359,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",39.0,0.09953,0.21,58.0,ml,quarry blast,"M 1.8 Quarry Blast - 4 km NNW of Boron, CA"
4,1.60,"18 km ENE of Tilden, Texas",1726615118477,1726663421747,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",tx2024siuz,",",tx,",",origin,phase-data,",20.0,0.10000,0.50,71.0,ml,earthquake,"M 1.6 - 18 km ENE of Tilden, Texas"


### (Optional) Write Data to CSV

In [8]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>